Installing the Necessary Libraries

In [4]:
pip install google-generativeai chromadb pandas pyPDF python-dotenv streamlit requests

Note: you may need to restart the kernel to use updated packages.


Importing the Necessary Libraries 

In [5]:
import os
from dotenv import load_dotenv
from pprint import pprint

import pandas as pd 

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

import google.generativeai as genai

from IPython.display import Markdown

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Checking Generative AI Version
genai.__version__

'0.8.3'

In [7]:
load_dotenv() #Loading Virtual Environment 

api_key = os.getenv('GEMINI_API_KEY') #Utilizing API Key 
genai.configure(api_key=api_key)

In [8]:
import json 
with open('./data/data.json') as f:
    data = json.load(f) #Importing Data

In [9]:
pprint(data) #Checking Data Format

[{'Additional Information': "Enjoy panoramic views of Dubai from the world's "
                            'tallest building. Pre-booked tickets include '
                            'access to indoor observation decks with '
                            'floor-to-ceiling glass walls and an open-air '
                            'terrace. Telescopes are provided for a closer '
                            'look at the cityscape.',
  'Attraction Name': 'Burj Khalifa: At The Top (Levels 124 and 125) Admission '
                     'Ticket',
  'Deal (Total Savings)': 'Save up to 10%',
  'Landmark Name': 'Burj Khalifa',
  'Location': 'Downtown Dubai',
  'Price (Per Adult)': 56,
  'Source': 'Groupon',
  'Source Link': 'https://www.groupon.com'},
 {'Additional Information': 'Explore one of the largest suspended aquariums in '
                            'the world, home to thousands of aquatic animals. '
                            'The ticket includes access to the Underwater Zoo '
         

Making JSON file into String for better vector embedding 

In [10]:
import json

# Load the JSON file
with open("./data/data.json", "r") as file:
    data = json.load(file)

documents = []

for item in data:
    entry = ""
    if item.get('Attraction Name', '') != '':
        entry += f"Attraction Name: {item['Attraction Name']}\n"

    if item.get('Deal (Total Savings)', '') != '':
        entry += f"Deal (Total Savings): {item['Deal (Total Savings)']}\n"

    if item.get('Location', '') != '':
        entry += f"Location: {item['Location']}\n"

    if item.get('Price (Per Adult)', '') != '':
        entry += f"Price (Per Adult): {item['Price (Per Adult)']}\n"

    if item.get('Source', '') != '':
        entry += f"Source: {item['Source']}\n"

    if item.get('Source Link', '') != '':
        entry += f"Source Link: {item['Source Link']}\n"

    if item.get('Additional Information', '') != '':
        entry += f"Additional Information: {item['Additional Information']}\n"
    
    if item.get('Price(In AED)', '') != '':
        entry += f"Price_AED {item['Price_AED']}\n"
    
    if item.get('Price(In USD)', '') != '':
        entry += f"Price_USD {item['Price_USD']}"
    
    if item.get('Landmark Name', '') != '':
        entry += f"Landmark Name: {item['Landmark Name']}\n"
    documents.append(entry)

print(len(documents))


113


In [11]:
pprint(documents[0])

('Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission '
 'Ticket\n'
 'Deal (Total Savings): Save up to 10%\n'
 'Location: Downtown Dubai\n'
 'Price (Per Adult): 56\n'
 'Source: Groupon\n'
 'Source Link: https://www.groupon.com\n'
 "Additional Information: Enjoy panoramic views of Dubai from the world's "
 'tallest building. Pre-booked tickets include access to indoor observation '
 'decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes '
 'are provided for a closer look at the cityscape.\n'
 'Landmark Name: Burj Khalifa\n')


Creating Embedding Function 

In [12]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = 'models/embedding-001'
        # for better results, try to provide a title for each input if the corpus is covering a lot of domains
        title = "Dubai's First AI Tourist Guide"

        return genai.embed_content(
            model=model,
            content=input,
            task_type="retrieval_document",
            title=title)["embedding"]

In [13]:
import time
from tqdm import tqdm

Creating Chroma DB 

In [14]:
def create_chroma_db(documents, name):
    chroma_client = chromadb.PersistentClient(path="../database/")

    db = chroma_client.get_or_create_collection(
        name=name, embedding_function=GeminiEmbeddingFunction())

    initiali_size = db.count()
    for i, d in tqdm(enumerate(documents), total=len(documents), desc="Creating Chroma DB"):
        db.add(
            documents=d,
            ids=str(i + initiali_size)
        )
        time.sleep(0.5)
    return db


def get_chroma_db(name):
    chroma_client = chromadb.PersistentClient(path="../database/")
    return chroma_client.get_collection(name=name, embedding_function=GeminiEmbeddingFunction())

In [15]:
db = create_chroma_db(documents, "sme_db")
db.count()

Creating Chroma DB: 100%|██████████| 113/113 [02:05<00:00,  1.11s/it]


113

In [16]:
def get_relevant_passages(query, db, n_results=5):
    passages = db.query(query_texts=[query], n_results=n_results)[
        'documents'][0]
    return passages

In [17]:
from IPython.display import Markdown

# Define the question and passages
question = "Can you recommend some deals for Burj Khalifa?"

passages = documents

def make_prompt(query, relevant_passage):
    escaped = relevant_passage.replace("'", "").replace('"', "")
    prompt = f"""You are a chatbot that specializes in helping users find deals for tourist attractions in Dubai.\n
    User's question: {query}\n
    Context information about available deals:\n {escaped}\n
    To assist with the user's question, please follow these steps:\n
    1. Parse the user's query to identify the tourist attraction mentioned (e.g., "Burj Khalifa"). Search through the provided data, including the 'landmark name' column, for matches using exact or partial keywords.\n
    2. For each matching entry, extract and display the following details:
       - Landmark Name\n
       - Deal Description\n
       - Price (in AED and converted to USD, assuming a fixed exchange rate provided in the dataset or calculated dynamically).\n
    3. If deals for the requested attraction are available, list the most relevant options with clear and concise information.\n
    4. If no deals are available for the specified attraction, inform the user politely and suggest nearby or related attractions with available deals.\n
    5. Ask the user about their preferences (e.g., type of activity, budget range, or specific interests) to tailor recommendations.\n
    6. Include practical information such as operating hours, recommended visit times, or proximity to other attractions when available.\n
    Your response should be friendly, professional, and helpful, catering to tourists seeking great experiences in Dubai.\n
    """
    return prompt

def convert_passages_to_list(passages):
    context = ""

    for passage in passages:
        context += passage + "\n"

    return context

# Generate the prompt
prompt = make_prompt(question, convert_passages_to_list(passages))

# Render the prompt as Markdown (for Jupyter Notebooks or compatible environments)
Markdown(prompt)


You are a chatbot that specializes in helping users find deals for tourist attractions in Dubai.

    User's question: Can you recommend some deals for Burj Khalifa?

    Context information about available deals:
 Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Other Landmark

Attraction Name: Burj Khalifa: At The Top (Levels 124 and 125) Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Downtown Dubai
Price (Per Adult): 56
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Enjoy panoramic views of Dubai from the worlds tallest building. Pre-booked tickets include access to indoor observation decks with floor-to-ceiling glass walls and an open-air terrace. Telescopes are provided for a closer look at the cityscape.
Landmark Name: Burj Khalifa

Attraction Name: Dubai Aquarium and Underwater Zoo with Penguin Cove Tickets
Deal (Total Savings): Save up to 15%
Location: Dubai Mall
Price (Per Adult): 42
Source: Tripadvisor
Source Link: https://www.tripadvisor.com
Additional Information: Explore one of the largest suspended aquariums in the world, home to thousands of aquatic animals. The ticket includes access to the Underwater Zoo and Penguin Cove, offering an interactive experience with marine life.
Landmark Name: Dubai Aquarium

Attraction Name: Desert Safari with BBQ Dinner and Live Entertainment
Deal (Total Savings): Save up to 20%
Location: Dubai Desert
Price (Per Adult): 65
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of dune bashing in the Dubai desert, followed by a traditional BBQ dinner and live entertainment at a Bedouin-style camp. Activities include camel riding, sandboarding, henna painting, and belly dancing performances.
Landmark Name: Dubai Desert

Attraction Name: Atlantis Aquaventure Waterpark Admission
Deal (Total Savings): Save up to 15%
Location: Palm Jumeirah
Price (Per Adult): 75
Source: Klook
Source Link: https://www.klook.com
Additional Information: Enjoy a full day of excitement at the largest waterpark in the Middle East, featuring thrilling water slides, a private beach, and access to the Lost Chambers Aquarium. Suitable for all ages with dedicated childrens play areas.
Landmark Name: Atlantis Aquaventure Waterpark

Attraction Name: Museum of the Future Tickets
Deal (Total Savings): Save up to 10%
Location: Trade Centre
Price (Per Adult): 42
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Visit Dubais latest architectural marvel and explore innovative exhibits that offer a glimpse into the future of science, technology, and innovation. The museum provides an immersive experience for visitors of all ages.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Tickets
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Tiqets
Source Link: https://www.tiqets.com
Additional Information: Discover Dubais past, present, and future through interactive galleries and enjoy panoramic views of the city from the sky deck of this iconic architectural landmark.
Landmark Name: Other Landmark

Attraction Name: Dubai Miracle Garden Entry Tickets
Deal (Total Savings): Save up to 10%
Location: Al Barsha South
Price (Per Adult): 20
Source: Headout
Source Link: https://www.headout.com
Additional Information: Explore the worlds largest natural flower garden, featuring over 150 million flowers arranged in stunning designs and themes. A family-friendly attraction open from November to April.
Landmark Name: Other Landmark

Attraction Name: Dubai Marina Dhow Cruise with Dinner
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 55
Source: Official Website
Source Link: https://www.burjkhalifa.ae
Additional Information: Enjoy a relaxing evening aboard a traditional dhow cruise along Dubai Marina, complete with a buffet dinner, live music, and stunning views of the citys illuminated skyline.
Landmark Name: Other Landmark

Attraction Name: IMG Worlds of Adventure Tickets
Deal (Total Savings): Save up to 10%
Location: Sheikh Mohammed Bin Zayed Road
Price (Per Adult): 75
Source: Local Vendor
Source Link: https://www.localvendor.com
Additional Information: Experience the worlds largest indoor theme park, featuring thrilling rides and attractions based on popular Cartoon Network characters and Marvel superheroes. Suitable for all age groups.
Landmark Name: Other Landmark

Attraction Name: Dubai Dolphinarium Tickets
Deal (Total Savings): Save up to 10%
Location: Riyadh Street
Price (Per Adult): 30
Source: Expedia
Source Link: https://www.expedia.com
Additional Information: Enjoy an entertaining dolphin and seal show showcasing the incredible talents of these marine animals. Optional experiences include swimming with dolphins and photo sessions.
Landmark Name: Dubai Dolphinarium

Attraction Name: Burj Al Arab Guided Tour
Deal (Total Savings): Save up to 15%
Location: Jumeirah Beach Road
Price (Per Adult): 80
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience a guided tour of the worlds most luxurious hotel. Explore its opulent interiors, learn about its architecture, and enjoy stunning views of the Arabian Gulf.
Landmark Name: Burj Al Arab

Attraction Name: Jumeirah Beach Water Sports Package
Deal (Total Savings): Save up to 20%
Location: Jumeirah Beach
Price (Per Adult): 50
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Enjoy a variety of water sports at Jumeirah Beach, including jet skiing, parasailing, and paddleboarding. Packages are customizable and include equipment rental.
Landmark Name: Jumeirah Beach

Attraction Name: Palm Jumeirah Monorail Ride
Deal (Total Savings): Save up to 10%
Location: Palm Jumeirah
Price (Per Adult): 15
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Travel through Palm Jumeirah on the iconic monorail and enjoy stunning views of the Arabian Gulf and Atlantis, The Palm.
Landmark Name: Palm Jumeirah

Attraction Name: Skydive Dubai Tandem Jump
Deal (Total Savings): Save up to 5%
Location: Skydive Dubai, Palm Jumeirah
Price (Per Adult): 450
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience an adrenaline-pumping tandem skydive over Palm Jumeirah or the Dubai Desert. Includes photos and video of your jump.
Landmark Name: Skydive Dubai

Attraction Name: Museum of the Future Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Sheikh Zayed Road
Price (Per Adult): 40
Source: Groupon
Source Link: https://www.groupon.com
Additional Information: Discover cutting-edge innovations and futuristic exhibits at Dubais iconic Museum of the Future. Tickets include access to all exhibits and interactive displays.
Landmark Name: Museum of the Future

Attraction Name: Dubai Frame Admission Ticket
Deal (Total Savings): Save up to 10%
Location: Zabeel Park
Price (Per Adult): 15
Source: Klook
Source Link: https://www.klook.com
Additional Information: Visit Dubai Frame, a landmark offering panoramic views of both old and new Dubai. Enjoy exhibits showcasing the citys transformation.
Landmark Name: Dubai Frame

Attraction Name: Atlantis Dubai Dolphin Encounter
Deal (Total Savings): Save up to 20%
Location: Atlantis The Palm
Price (Per Adult): 150
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Interact with friendly dolphins in a shallow lagoon. Perfect for all ages, this experience includes professional photos and access to Atlantis facilities.
Landmark Name: Atlantis Dubai

Attraction Name: Dubai Mall Shopping Experience
Deal (Total Savings): Save up to 15%
Location: Downtown Dubai
Price (Per Adult): Free entry; guided tours from $25
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Explore the worlds largest shopping mall with over 1,200 shops, an aquarium, an ice rink, and the iconic Dubai Fountain show.
Landmark Name: Dubai Mall

Attraction Name: Mall of the Emirates Ski Package
Deal (Total Savings): Save up to 20%
Location: Al Barsha
Price (Per Adult): 60
Source: Viator
Source Link: https://www.viator.com
Additional Information: Enjoy skiing or snowboarding at Ski Dubai, located inside the Mall of the Emirates. Includes gear rental and access to the snow park.
Landmark Name: Mall of the Emirates

Attraction Name: Dubai Festival City Shopping and Laser Show
Deal (Total Savings): Free Laser Show
Location: Festival City
Price (Per Adult): Free entry; shopping as per choice
Source: Dubai Festival City
Source Link: https://www.dubaifestivalcity.com
Additional Information: Shop at luxury stores and enjoy the spectacular IMAGINE laser and fountain show, a must-see attraction.
Landmark Name: Dubai Festival City Mall

Attraction Name: Dubai Marina Jet Ski Tour
Deal (Total Savings): Save up to 15%
Location: Dubai Marina
Price (Per Adult): 100
Source: Klook
Source Link: https://www.klook.com
Additional Information: Embark on an exhilarating jet ski ride around Dubai Marina. Witness iconic landmarks like Atlantis, The Palm, and Burj Al Arab.
Landmark Name: Dubai Marina

Attraction Name: Parasailing Over Jumeirah Beach
Deal (Total Savings): Save up to 10%
Location: Jumeirah Beach
Price (Per Adult): 85
Source: Viator
Source Link: https://www.viator.com
Additional Information: Experience the thrill of parasailing over Jumeirah Beach with breathtaking views of the coastline and city skyline.
Landmark Name: Jumeirah Beach

Attraction Name: Banana Boat Ride in Dubai
Deal (Total Savings): Save up to 20%
Location: Jumeirah Beach
Price (Per Adult): 30
Source: GetYourGuide
Source Link: https://www.getyourguide.com
Additional Information: Enjoy a fun-filled banana boat ride with friends and family along Dubais stunning coastline.
Landmark Name: Jumeirah Beach



    To assist with the user's question, please follow these steps:

    1. Parse the user's query to identify the tourist attraction mentioned (e.g., "Burj Khalifa"). Search through the provided data, including the 'landmark name' column, for matches using exact or partial keywords.

    2. For each matching entry, extract and display the following details:
       - Landmark Name

       - Deal Description

       - Price (in AED and converted to USD, assuming a fixed exchange rate provided in the dataset or calculated dynamically).

    3. If deals for the requested attraction are available, list the most relevant options with clear and concise information.

    4. If no deals are available for the specified attraction, inform the user politely and suggest nearby or related attractions with available deals.

    5. Ask the user about their preferences (e.g., type of activity, budget range, or specific interests) to tailor recommendations.

    6. Include practical information such as operating hours, recommended visit times, or proximity to other attractions when available.

    Your response should be friendly, professional, and helpful, catering to tourists seeking great experiences in Dubai.

    

Response Generation

In [18]:
model = genai.GenerativeModel('gemini-pro')

In [19]:
answer = model.generate_content(prompt)
Markdown(answer.text)

I found several deals for Burj Khalifa At The Top (Levels 124 and 125) Admission Ticket:

- **Groupon:** Save up to 10%, priced at AED 200 (USD 54) per adult.
- **Viator:** Save up to 12%, priced at AED 210 (USD 57) per adult, and includes skip-the-line access.
- **Klook:** Save up to 15%, priced at AED 196 (USD 53) per adult, and offers instant confirmation.

All prices are subject to availability and may vary depending on the date and time of your visit. To check for the latest deals and availability, I recommend visiting the official websites of these providers or searching online travel agencies.

Would you like to know more about any of these deals or need recommendations for other attractions in Dubai?